In [39]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, Tool, tool
from langgraph.graph import StateGraph, END
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage, FunctionMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt.tool_executor import ToolExecutor
from langgraph.prebuilt import ToolInvocation
from langchain.tools import format_tool_to_openai_function
from typing import TypedDict, Sequence
import re
import requests
from jupyter_server.serverapp import list_running_servers
import json

import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key
llm = ChatOpenAI(api_key=api_key, model="gpt-4", temperature=0, streaming=True)

class AgentState(TypedDict):
    messages: Sequence[BaseMessage]
    notebook_path: str  # Add notebook path


<button data-commandLinker-command="notebook:run-cell" href="#">TEST</button>

In [40]:
def get_jupyter_server():
    """
    Returns the first available Jupyter server URL and token.
    """
    servers = list(list_running_servers())
    if not servers:
        return None
    
    # Return the first running Jupyter server
    return servers[0]  # Contains `notebook_dir`, `url`, `token`, etc.

server = get_jupyter_server()

In [41]:
server

{'base_url': '/',
 'hostname': 'localhost',
 'password': False,
 'pid': 61811,
 'port': 8888,
 'root_dir': '/Users/abigaylemercer/Desktop/Masters',
 'secure': False,
 'sock': '',
 'token': '33ba6d641485faeb59218a6894e448d000d1d2dd71cedbde',
 'url': 'http://localhost:8888/',
 'version': '2.15.0'}

In [42]:
def get_active_notebook():
    """
    Retrieves the currently active notebook's API path.
    """
    if not server:
        return None
    
    base_url = server["url"]
    token = server["token"]
    
    # Example: Call Jupyter Server API to list running sessions
    try:
        response = requests.get(
            f"{base_url}api/sessions",
            headers={"Authorization": f"token {token}"}
        )
        response.raise_for_status()
        
        sessions = response.json()
        
        # Find the first notebook session
        for session in sessions:
            if session.get("type") == "notebook":
                return base_url, session["path"]

        return None  # No active notebook found

    except requests.RequestException as e:
        print(f"Error retrieving active notebook: {e}")
        return None

In [46]:
print(get_active_notebook())

('http://localhost:8888/', 'thesis-explorations/thesis-explorations/test_data/test_book.ipynb')


In [34]:
@tool("run_current_cell", return_direct=True)
def run_current_cell():
    """
    Runs the currently selected cell in the active Jupyter Notebook.
    """
    notebook_info = get_active_notebook()
    if not notebook_info:
        return "No active Jupyter notebook found."

    base_url, notebook_path = notebook_info
    api_url = f"{base_url}api/contents/{notebook_path}/run"

    try:
        headers = {"Authorization": f"token {server['token']}"}
        response = requests.post(api_url, headers=headers)

        if response.status_code == 200:
            return "Cell executed successfully."
        else:
            return f"Error executing cell: {response.text}"

    except Exception as e:
        return f"Error interacting with Jupyter: {str(e)}"

In [35]:
# Step 3: Create the Tool Executor
tools = [run_current_cell]
tool_executor = ToolExecutor(tools)
functions = [format_tool_to_openai_function(t) for t in tools]
model = llm.bind_functions(functions)

# Step 4: Define the Agent
def agent(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": messages + [response]}

def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    return "continue" if "function_call" in last_message.additional_kwargs else "end"

def call_tool(state):
    messages = state["messages"]
    last_message = messages[-1]
    notebook_path = state["notebook_path"]

    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input={"notebook_path": notebook_path}  # Pass notebook path
    )

    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=action.tool)
    return {"messages": messages + [function_message], "notebook_path": notebook_path}

# Step 5: Build Workflow
workflow = StateGraph(AgentState)
workflow.add_node("agent", agent)
workflow.add_node("action", call_tool)
workflow.set_entry_point("agent")
workflow.add_conditional_edges("agent", should_continue, {"continue": "action", "end": END})
workflow.add_edge("action", "agent")
app = workflow.compile()

# Export the agent
def get_agent():
    return app


/var/folders/3h/z8r254ts5r58qphcnd1h46xw0000gn/T/ipykernel_58876/2634666359.py:3: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


In [36]:
notebook_info = get_active_notebook()


In [37]:
notebook_info

('http://localhost:8888/', 'test.ipynb')

In [38]:
if notebook_info:
    base_url, notebook_path = notebook_info

    # Start agent with the notebook context
    inputs = {
        "messages": [SystemMessage(content="You are an AI assistant for Jupyter Notebooks."),
                     HumanMessage(content="Run the current cell.")],
        "notebook_path": notebook_path
    }

    response = app.invoke(inputs)
    print(response)
else:
    print("No active Jupyter notebooks found.")

/var/folders/3h/z8r254ts5r58qphcnd1h46xw0000gn/T/ipykernel_58876/2634666359.py:23: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  action = ToolInvocation(


{'messages': [SystemMessage(content='You are an AI assistant for Jupyter Notebooks.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Run the current cell.', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{}', 'name': 'run_current_cell'}}, response_metadata={'finish_reason': 'function_call'}, id='run-adc5239f-9f33-41db-8cae-511c20480682-0'), FunctionMessage(content='Error executing cell: {"message": "No such directory: test.ipynb/run", "reason": null}', additional_kwargs={}, response_metadata={}, name='run_current_cell'), AIMessage(content='I\'m sorry, but there seems to be an error executing the cell. The directory or file "test.ipynb/run" does not exist. Please make sure the file and directory are correct.', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-e4e54eae-bdea-4553-bbcb-7c72e3b67dbf-0')], 'notebook_path': 'test.ipynb'}
